# Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from scipy.stats import norm

# Setup

In [2]:
def recalib(forecast): # 2000 looks best
    return forecast+.0055*(2*(forecast-.5))**17

In [3]:
NBA_data= pd.read_csv("nbaallelo.csv")
NBA_data

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,40.294830,NYK,Knicks,68,1300.0000,1306.7233,H,L,0.640065,NaN
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,41.705170,TRH,Huskies,66,1300.0000,1293.2767,A,W,0.359935,NaN
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,42.012257,NYK,Knicks,47,1306.7233,1297.0712,H,W,0.631101,NaN
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,40.692783,CHS,Stags,63,1300.0000,1309.6521,A,L,0.368899,NaN
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,38.864048,WSC,Capitols,50,1300.0000,1320.3811,H,L,0.640065,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126309,63155,201506110CLE,NBA,0,2015,6/11/2015,100,1,CLE,Cavaliers,...,60.309792,GSW,Warriors,103,1790.9591,1809.9791,H,L,0.546572,NaN
126310,63156,201506140GSW,NBA,0,2015,6/14/2015,102,1,GSW,Warriors,...,68.013329,CLE,Cavaliers,91,1704.3949,1700.7391,H,W,0.765565,NaN
126311,63156,201506140GSW,NBA,1,2015,6/14/2015,101,1,CLE,Cavaliers,...,60.010067,GSW,Warriors,104,1809.9791,1813.6349,A,L,0.234435,NaN
126312,63157,201506170CLE,NBA,0,2015,6/16/2015,102,1,CLE,Cavaliers,...,59.290245,GSW,Warriors,105,1813.6349,1822.2881,H,L,0.481450,NaN


In [4]:
NBA_data.columns

Index(['gameorder', 'game_id', 'lg_id', '_iscopy', 'year_id', 'date_game',
       'seasongame', 'is_playoffs', 'team_id', 'fran_id', 'pts', 'elo_i',
       'elo_n', 'win_equiv', 'opp_id', 'opp_fran', 'opp_pts', 'opp_elo_i',
       'opp_elo_n', 'game_location', 'game_result', 'forecast', 'notes'],
      dtype='object')

In [5]:
NBA_data['rc_forecast'] = NBA_data.apply(lambda x: recalib(x['forecast']), axis=1)
NBA_data['results'] = (NBA_data['game_result'] == 'W')*1

In [6]:
NBA_var=np.var(NBA_data['elo_i'])
NBA_mean=np.mean(NBA_data[NBA_data['team_id']=='NYK']['elo_i'])

print(NBA_mean, NBA_var) #/(126314**.5)

1497.6156040908302 12575.267726994409


In [7]:
NBA_data[['elo_i', 'opp_elo_i', 'forecast', 'rc_forecast', 'results']]

,elo_i,opp_elo_i,forecast,rc_forecast,results
0,1300.0000,1300.0000,0.640065,0.640065,0
1,1300.0000,1300.0000,0.359935,0.359935,1
2,1300.0000,1306.7233,0.631101,0.631101,1
3,1306.7233,1300.0000,0.368899,0.368899,0
4,1300.0000,1300.0000,0.640065,0.640065,0
...,...,...,...,...,...
126309,1723.4149,1790.9591,0.546572,0.546572,0
126310,1809.9791,1704.3949,0.765565,0.765566,1
126311,1704.3949,1809.9791,0.234435,0.234434,0
126312,1700.7391,1813.6349,0.481450,0.481450,0


In [8]:
regdf1=NBA_data[['game_id','forecast', 'rc_forecast']]
model1 = smf.ols(formula='rc_forecast ~ forecast', data=regdf1).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            rc_forecast   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 6.951e+11
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:27:28   Log-Likelihood:             9.9501e+05
No. Observations:              126314   AIC:                        -1.990e+06
Df Residuals:                  126312   BIC:                        -1.990e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -7.416e-05   6.53e-07   -113.561      0.000   -7.54e-05   -7.29e-05
forecast       1.0001    1.2e-06   8.34e+05      0.000       1.000       1.000
==============================================================================
Omnibus:                    54835.578   Durbin-Watson:                   2.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        176826740.933
Skew:                          -0.000   Prob(JB):                         0.00
Kurtosis:                     186.296   Cond. No.                         5.85
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 538 Model Test Results

In [9]:
regdf2=NBA_data[['game_id','forecast', 'results']]
model2 = smf.ols(formula='forecast ~ results', data=regdf2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               forecast   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                 2.864e+04
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:27:28   Log-Likelihood:                 27686.
No. Observations:              126314   AIC:                        -5.537e+04
Df Residuals:                  126312   BIC:                        -5.535e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4075      0.001    526.892      0.000       0.406       0.409
results        0.1851      0.001    169.227      0.000       0.183       0.187
==============================================================================
Omnibus:                     6278.891   Durbin-Watson:                   3.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2485.259
Skew:                          -0.000   Prob(JB):                         0.00
Kurtosis:                       2.313   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
log_reg1 = sm.Logit(regdf2['results'], regdf2['forecast']).fit() 
log_reg1.summary()

Optimization terminated successfully.
         Current function value: 0.678589
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126313
Method:                           MLE   Df Model:                            0
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                 0.02100
Time:                        18:27:28   Log-Likelihood:                -85715.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
forecast       0.6341      0.011     59.958      0.000       0.613       0.655
==============================================================================
"""

# RC Test Results

In [11]:
regdf3=NBA_data[['game_id','rc_forecast', 'results']]
model3 = smf.ols(formula='rc_forecast ~ results', data=regdf3).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            rc_forecast   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                 2.864e+04
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:27:28   Log-Likelihood:                 27667.
No. Observations:              126314   AIC:                        -5.533e+04
Df Residuals:                  126312   BIC:                        -5.531e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4074      0.001    526.796      0.000       0.406       0.409
results        0.1851      0.001    169.227      0.000       0.183       0.187
==============================================================================
Omnibus:                     6256.299   Durbin-Watson:                   3.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2479.841
Skew:                          -0.000   Prob(JB):                         0.00
Kurtosis:                       2.314   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
log_reg2 = sm.Logit(regdf3['results'], regdf3['rc_forecast']).fit() 
log_reg2.summary()

Optimization terminated successfully.
         Current function value: 0.678585
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126313
Method:                           MLE   Df Model:                            0
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                 0.02101
Time:                        18:27:28   Log-Likelihood:                -85715.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                       nan
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
rc_forecast     0.6342      0.011     59.966      0.000       0.613       0.655
===============================================================================
"""

# Weighted Mix

In [13]:
a=.1
NBA_data['w_forecast']= NBA_data['rc_forecast']*a+ NBA_data['forecast']*(1-a)

In [14]:
regdf4=NBA_data[['game_id','w_forecast', 'results','rc_forecast','forecast']]
model4 = smf.ols(formula='w_forecast ~ results', data=regdf4).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             w_forecast   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                 2.864e+04
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:27:28   Log-Likelihood:                 27684.
No. Observations:              126314   AIC:                        -5.536e+04
Df Residuals:                  126312   BIC:                        -5.534e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4075      0.001    526.882      0.000       0.406       0.409
results        0.1851      0.001    169.227      0.000       0.183       0.187
==============================================================================
Omnibus:                     6276.638   Durbin-Watson:                   3.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2484.719
Skew:                          -0.000   Prob(JB):                         0.00
Kurtosis:                       2.313   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
log_reg3 = sm.Logit(regdf3['results'], regdf4[['forecast','rc_forecast']]).fit() 
log_reg3.summary()

C:\Users\rhnma\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


         Current function value: inf
         Iterations: 35


C:\Users\rhnma\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1852: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
C:\Users\rhnma\Anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\rhnma\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126312
Method:                           MLE   Df Model:                            1
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                    -inf
Time:                        18:27:29   Log-Likelihood:                   -inf
converged:                      False   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                     1.000
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
forecast     2.466e+06   3.67e+04     67.194      0.000    2.39e+06    2.54e+06
rc_forecast -2.466e+06   3.67e+04    -67.194      0.000   -2.54e+06   -2.39e+06
===============================================================================
"""

In [16]:
log_reg4 = sm.Logit(regdf4['results'], regdf4['w_forecast']).fit() 
log_reg4.summary()

Optimization terminated successfully.
         Current function value: 0.678588
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126313
Method:                           MLE   Df Model:                            0
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                 0.02100
Time:                        18:27:29   Log-Likelihood:                -85715.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
w_forecast     0.6341      0.011     59.959      0.000       0.613       0.655
==============================================================================
"""

# Binned Testing 538

In [17]:
bindff=NBA_data[['forecast','results']]
bins= np.arange(0,1.01,.05)
labels= np.arange(0,100,5)
print(bins)
bindff['binned']=pd.cut(bindff['forecast'],bins=bins,labels=labels)
bindff['binned']=bindff['binned'].fillna(0)
bindff

[0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]


C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,forecast,results,binned
0,0.640065,0,60
1,0.359935,1,35
2,0.631101,1,60
3,0.368899,0,35
4,0.640065,0,60
...,...,...,...
126309,0.546572,0,50
126310,0.765565,1,75
126311,0.234435,0,20
126312,0.481450,0,45


In [18]:
bin_compf=[]
for i in range(len(bins)-1):
    binaver= np.mean(bindff['results'][bindff['binned']==labels[i]])
    bin_compf.append([(bins[i]+bins[i+1])/2,binaver])
bin_compf

[[0.025, 0.031055900621118012],
 [0.07500000000000001, 0.07574094401756312],
 [0.125, 0.1243680485338726],
 [0.17500000000000002, 0.17873542917696927],
 [0.225, 0.2273752466873414],
 [0.275, 0.27632074287865477],
 [0.32500000000000007, 0.3236698878151727],
 [0.375, 0.37402207694780837],
 [0.42500000000000004, 0.43101182654402104],
 [0.475, 0.48166611277279275],
 [0.525, 0.5183338872272073],
 [0.5750000000000001, 0.568988173455979],
 [0.625, 0.6259779230521916],
 [0.675, 0.6763301121848273],
 [0.7250000000000001, 0.7236792571213452],
 [0.775, 0.7726247533126586],
 [0.8250000000000001, 0.8212645708230307],
 [0.875, 0.8756319514661274],
 [0.925, 0.9242590559824369],
 [0.9750000000000001, 0.968944099378882]]

# Binned Testing

In [19]:
bindf=NBA_data[['rc_forecast','results']]

In [20]:
bins= np.arange(0,1.01,.05)
labels= np.arange(0,100,5)
print(bins)
bindf['binned']=pd.cut(bindf['rc_forecast'],bins=bins,labels=labels)

[0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]


C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [21]:
bindf['binned']=bindf['binned'].fillna(0)

C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
bindf

,rc_forecast,results,binned
0,0.640065,0,60
1,0.359935,1,35
2,0.631101,1,60
3,0.368899,0,35
4,0.640065,0,60
...,...,...,...
126309,0.546572,0,50
126310,0.765566,1,75
126311,0.234434,0,20
126312,0.481450,0,45


In [23]:
#set(bindf['binned'])

In [24]:
bin_comp=[]
for i in range(len(bins)-1):
    binaver= np.mean(bindf['results'][bindf['binned']==labels[i]])
    bin_comp.append([(bins[i]+bins[i+1])/2,binaver])

In [25]:
bin_comp

[[0.025, 0.02857142857142857],
 [0.07500000000000001, 0.07713498622589532],
 [0.125, 0.12401923563654771],
 [0.17500000000000002, 0.17879858657243816],
 [0.225, 0.2273752466873414],
 [0.275, 0.27632074287865477],
 [0.32500000000000007, 0.3236698878151727],
 [0.375, 0.37402207694780837],
 [0.42500000000000004, 0.43101182654402104],
 [0.475, 0.48166611277279275],
 [0.525, 0.5183338872272073],
 [0.5750000000000001, 0.568988173455979],
 [0.625, 0.6259779230521916],
 [0.675, 0.6763301121848273],
 [0.7250000000000001, 0.7236792571213452],
 [0.775, 0.7726247533126586],
 [0.8250000000000001, 0.8212014134275618],
 [0.875, 0.8759807643634523],
 [0.925, 0.9228650137741047],
 [0.9750000000000001, 0.9714285714285714]]

# Weighted Binned Testing

In [26]:
bindfw=NBA_data[['w_forecast','results']]
bins= np.arange(0,1.01,.05)
labels= np.arange(0,100,5)
print(bins)
bindfw['binned']=pd.cut(bindfw['w_forecast'],bins=bins,labels=labels)
bindfw['binned']=bindfw['binned'].fillna(0)
bindfw

[0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]


C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,w_forecast,results,binned
0,0.640065,0,60
1,0.359935,1,35
2,0.631101,1,60
3,0.368899,0,35
4,0.640065,0,60
...,...,...,...
126309,0.546572,0,50
126310,0.765565,1,75
126311,0.234435,0,20
126312,0.481450,0,45


In [27]:
bin_compw=[]
for i in range(len(bins)-1):
    binaver= np.mean(bindfw['results'][bindfw['binned']==labels[i]])
    bin_compw.append([(bins[i]+bins[i+1])/2,binaver])
bin_compw

[[0.025, 0.031055900621118012],
 [0.07500000000000001, 0.07616438356164383],
 [0.125, 0.1242094611687326],
 [0.17500000000000002, 0.17873542917696927],
 [0.225, 0.2273752466873414],
 [0.275, 0.27632074287865477],
 [0.32500000000000007, 0.3236698878151727],
 [0.375, 0.37402207694780837],
 [0.42500000000000004, 0.43101182654402104],
 [0.475, 0.48166611277279275],
 [0.525, 0.5183338872272073],
 [0.5750000000000001, 0.568988173455979],
 [0.625, 0.6259779230521916],
 [0.675, 0.6763301121848273],
 [0.7250000000000001, 0.7236792571213452],
 [0.775, 0.7726247533126586],
 [0.8250000000000001, 0.8212645708230307],
 [0.875, 0.8757905388312673],
 [0.925, 0.9238356164383562],
 [0.9750000000000001, 0.968944099378882]]

# End